In [4]:
# -*- coding: utf-8 -*-
"""
DISCLAIMER: 

This software is provided solely for educational and research purposes. 
It is not intended to provide investment advice, and no investment recommendations are made herein. 
The developers are not financial advisors and accept no responsibility for any financial decisions or losses resulting from the use of this software. 
Always consult a professional financial advisor before making any investment decisions.
"""

import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta # For technical indicators
from datetime import datetime, timedelta
from scipy.interpolate import interp1d
from scipy.stats import percentileofscore # For IV Percentile proxy

# --- Constants ---
TRADING_PERIODS_PER_YEAR = 252
MIN_AVG_VOLUME = 1_500_000
IV_RV_THRESHOLD = 0.80 # Threshold for IV/RV ratio (Adjusted from 0.64 for potentially more candidates)
IV_PERCENTILE_THRESHOLD = 30 # Threshold for IV Percentile (<= 30%)
HISTORY_PERIOD = "1y" # Data needed for TAs and historical RV
RV_WINDOW = 30 # Window for Realized Volatility calculation
FAST_SMA = 10
SLOW_SMA = 20
RSI_PERIOD = 14
RSI_OVERSOLD = 30
STOCH_K = 14
STOCH_D = 3
STOCH_SMOOTH_K = 3
STOCH_OVERSOLD = 20
MACD_FAST = 12
MACD_SLOW = 26
MACD_SIGNAL = 9
BBANDS_PERIOD = 20
BBANDS_STD = 2.0
BBANDS_BANDWIDTH_PCT_THRESHOLD = 15 # Bottom percentile for Bollinger Bandwidth squeeze detection


# --- Functions for Calculations ---

def yang_zhang(price_data, window=RV_WINDOW, trading_periods=TRADING_PERIODS_PER_YEAR, return_series=False):
    """
    Calculate realized volatility using the Yang–Zhang method.
    price_data must contain the columns: 'Open', 'High', 'Low', 'Close'
    Can return the full series or just the last value.
    """
    log_ho = np.log(price_data['High'] / price_data['Open'])
    log_lo = np.log(price_data['Low'] / price_data['Open'])
    log_co = np.log(price_data['Close'] / price_data['Open'])
    
    log_oc = np.log(price_data['Open'] / price_data['Close'].shift(1))
    log_oc_sq = log_oc ** 2
    
    log_cc = np.log(price_data['Close'] / price_data['Close'].shift(1))
    log_cc_sq = log_cc ** 2
    
    rs = log_ho * (log_ho - log_co) + log_lo * (log_lo - log_co)
    
    close_vol = log_cc_sq.rolling(window=window, min_periods=window).sum() / (window - 1.0)
    open_vol  = log_oc_sq.rolling(window=window, min_periods=window).sum() / (window - 1.0)
    window_rs = rs.rolling(window=window, min_periods=window).sum() / (window - 1.0)
    
    k = 0.34 / (1.34 + ((window + 1) / (window - 1)))
    
    # Clamp values to avoid sqrt of negative numbers if rs is negative
    variance = open_vol + k * close_vol + (1 - k) * window_rs
    variance = variance.clip(lower=0) # Ensure non-negative variance
    
    result = np.sqrt(variance) * np.sqrt(trading_periods)

    if return_series:
        return result
    else:
        # Ensure we have enough data to calculate the last value
        return result.iloc[-1] if not result.empty and len(result) >= window else np.nan


# --- Helper Functions ---

def get_atm_iv_for_exp(stock, exp_str, underlying_price):
    """
    Retrieve the ATM implied volatility for a given expiration date.
    Pass underlying_price to avoid repeated calls.
    """
    try:
        # Check if options exist for this expiry before fetching
        if exp_str not in stock.options:
            # print(f"DEBUG: Expiry {exp_str} not found in stock.options for {stock.ticker}")
            return None

        chain = stock.option_chain(exp_str)
        calls = chain.calls
        puts = chain.puts
        
        # Check if underlying price is available
        if underlying_price is None or np.isnan(underlying_price):
             print(f"DEBUG: Invalid underlying price for {stock.ticker}")
             return None

        # Ensure strikes are numeric
        calls['strike'] = pd.to_numeric(calls['strike'], errors='coerce')
        puts['strike'] = pd.to_numeric(puts['strike'], errors='coerce')
        calls = calls.dropna(subset=['strike'])
        puts = puts.dropna(subset=['strike'])

        if calls.empty or puts.empty:
            # print(f"DEBUG: Empty calls or puts for {stock.ticker} expiry {exp_str}")
            return None

        # Find the call and put with strike closest to the underlying price.
        calls['abs_diff'] = (calls['strike'] - underlying_price).abs()
        puts['abs_diff'] = (puts['strike'] - underlying_price).abs()
        
        # Handle cases where min() raises error on empty sequence after filtering
        if calls['abs_diff'].isna().all() or puts['abs_diff'].isna().all():
            # print(f"DEBUG: All strikes resulted in NaN differences for {stock.ticker} expiry {exp_str}")
            return None
            
        call_idx = calls['abs_diff'].idxmin()
        put_idx = puts['abs_diff'].idxmin()

        call_iv = calls.loc[call_idx, 'impliedVolatility']
        put_iv = puts.loc[put_idx, 'impliedVolatility']

        # Handle potential missing IV values even for ATM strikes
        if pd.isna(call_iv) or pd.isna(put_iv):
             # Try finding strikes *exactly* matching if possible or nearest available
             # print(f"DEBUG: NaN IV for ATM strike for {stock.ticker} expiry {exp_str}. Trying fallback.")
             # Basic fallback: use whichever is available, or average if both available but one was NaN before
             if pd.isna(call_iv) and not pd.isna(put_iv):
                 atm_iv = put_iv
             elif not pd.isna(call_iv) and pd.isna(put_iv):
                 atm_iv = call_iv
             elif not pd.isna(call_iv) and not pd.isna(put_iv): # Both were valid, average them.
                 atm_iv = (call_iv + put_iv) / 2.0
             else: # Both NaN
                 # print(f"DEBUG: Both call and put IV are NaN for ATM strike {stock.ticker} expiry {exp_str}")
                 return None
        else:
            atm_iv = (call_iv + put_iv) / 2.0
        
        # Basic validation
        if atm_iv is None or np.isnan(atm_iv) or atm_iv <= 0:
            # print(f"DEBUG: Invalid ATM IV calculated ({atm_iv}) for {stock.ticker} expiry {exp_str}")
            return None
            
        return atm_iv

    except Exception as e:
        # print(f"Error fetching/processing option chain for {stock.ticker} expiry {exp_str}: {e}")
        return None


def get_sp500_tickers():
    """
    Scrape S&P 500 tickers from Wikipedia.
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    try:
        tables = pd.read_html(url)
        sp500_table = tables[0]
        # Handle potential changes in column names
        symbol_col = 'Symbol' if 'Symbol' in sp500_table.columns else sp500_table.columns[0] # Assume first col if 'Symbol' isn't there
        tickers = sp500_table[symbol_col].tolist()
        # Replace characters not allowed in yfinance tickers
        tickers = [ticker.replace('.', '-') for ticker in tickers]
         # BF.B -> BF-B, BRK.B -> BRK-B (common examples)
        return tickers
    except Exception as e:
        print(f"Error fetching S&P 500 tickers: {e}")
        # Fallback list if Wikipedia fails
        return ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'TSLA', 'META', 'JPM', 'JNJ', 'V'] # Example subset


def get_current_price(stock_data):
    """
    Retrieve the current (latest available) closing price from historical data.
    """
    if stock_data is None or stock_data.empty:
       return np.nan
    return stock_data['Close'].iloc[-1]

def calculate_technical_indicators(df):
    """ Calculates technical indicators using pandas_ta """
    if df is None or df.empty:
        return df

    # Calculate SMAs
    df.ta.sma(length=FAST_SMA, append=True, col_names=(f'SMA_{FAST_SMA}'))
    df.ta.sma(length=SLOW_SMA, append=True, col_names=(f'SMA_{SLOW_SMA}'))

    # Calculate RSI
    df.ta.rsi(length=RSI_PERIOD, append=True, col_names=(f'RSI_{RSI_PERIOD}'))

    # Calculate Stochastics (%K, %D)
    stoch = df.ta.stoch(k=STOCH_K, d=STOCH_D, smooth_k=STOCH_SMOOTH_K, append=False)
    if stoch is not None and not stoch.empty:
        df[f'STOCHk_{STOCH_K}_{STOCH_D}_{STOCH_SMOOTH_K}'] = stoch.iloc[:, 0] # %K is usually first column
        df[f'STOCHd_{STOCH_K}_{STOCH_D}_{STOCH_SMOOTH_K}'] = stoch.iloc[:, 1] # %D is usually second column

    # Calculate MACD (MACD line, Histogram, Signal line)
    macd = df.ta.macd(fast=MACD_FAST, slow=MACD_SLOW, signal=MACD_SIGNAL, append=False)
    if macd is not None and not macd.empty:
        df[f'MACD_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}'] = macd.iloc[:, 0] # MACD Line
        df[f'MACDh_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}'] = macd.iloc[:, 1] # Histogram
        df[f'MACDs_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}'] = macd.iloc[:, 2] # Signal Line

    # Calculate Bollinger Bands (Upper, Mid, Lower, Bandwidth, %B)
    bbands = df.ta.bbands(length=BBANDS_PERIOD, std=BBANDS_STD, append=False)
    if bbands is not None and not bbands.empty:
        df[f'BBL_{BBANDS_PERIOD}_{BBANDS_STD}'] = bbands.iloc[:, 0] # Lower
        df[f'BBM_{BBANDS_PERIOD}_{BBANDS_STD}'] = bbands.iloc[:, 1] # Mid
        df[f'BBU_{BBANDS_PERIOD}_{BBANDS_STD}'] = bbands.iloc[:, 2] # Upper
        df[f'BBB_{BBANDS_PERIOD}_{BBANDS_STD}'] = bbands.iloc[:, 3] # Bandwidth
        df[f'BBP_{BBANDS_PERIOD}_{BBANDS_STD}'] = bbands.iloc[:, 4] # Percent B

    return df

def check_reversal_signals(df):
    """ Checks the latest data point for bullish reversal signals """
    signals = {}
    triggered_count = 0
    last = df.iloc[-1]
    prev = df.iloc[-2] if len(df) > 1 else None

    # Signal 1: RSI Oversold + Bullish Cross/Tick Up
    rsi_col = f'RSI_{RSI_PERIOD}'
    signals['RSI_Oversold_Cross'] = False
    if rsi_col in df.columns and prev is not None:
        if last[rsi_col] < RSI_OVERSOLD and last[rsi_col] > prev[rsi_col]:
             signals['RSI_Oversold_Cross'] = True
             triggered_count += 1
        # Optional: Also consider crossing up *through* the threshold
        # elif prev[rsi_col] < RSI_OVERSOLD and last[rsi_col] >= RSI_OVERSOLD:
        #     signals['RSI_Oversold_Cross'] = True
        #     triggered_count += 1


    # Signal 2: Stochastic Oversold Bullish Cross
    stoch_k_col = f'STOCHk_{STOCH_K}_{STOCH_D}_{STOCH_SMOOTH_K}'
    stoch_d_col = f'STOCHd_{STOCH_K}_{STOCH_D}_{STOCH_SMOOTH_K}'
    signals['Stochastic_Oversold_Cross'] = False
    if stoch_k_col in df.columns and stoch_d_col in df.columns and prev is not None:
       if (last[stoch_k_col] < STOCH_OVERSOLD and
           last[stoch_d_col] < STOCH_OVERSOLD and
           last[stoch_k_col] > last[stoch_d_col] and # Current K > D
           prev[stoch_k_col] <= prev[stoch_d_col]): # Previous K <= D
            signals['Stochastic_Oversold_Cross'] = True
            triggered_count += 1

    # Signal 3: MACD Histogram Flip to Positive
    macd_h_col = f'MACDh_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}'
    signals['MACD_Histogram_Flip'] = False
    if macd_h_col in df.columns and prev is not None:
        if last[macd_h_col] > 0 and prev[macd_h_col] <= 0:
            signals['MACD_Histogram_Flip'] = True
            triggered_count += 1

    # Signal 4: Fast SMA Bullish Crossover Slow SMA
    sma_f_col = f'SMA_{FAST_SMA}'
    sma_s_col = f'SMA_{SLOW_SMA}'
    signals['SMA_Bullish_Cross'] = False
    if sma_f_col in df.columns and sma_s_col in df.columns and prev is not None:
        if last[sma_f_col] > last[sma_s_col] and prev[sma_f_col] <= prev[sma_s_col]:
            signals['SMA_Bullish_Cross'] = True
            triggered_count += 1

    # Signal 5: Bollinger Band Squeeze and Breakout (Price closes above Upper Band after low Bandwidth)
    bbu_col = f'BBU_{BBANDS_PERIOD}_{BBANDS_STD}'
    bbb_col = f'BBB_{BBANDS_PERIOD}_{BBANDS_STD}'
    close_col = 'Close'
    signals['Bollinger_Squeeze_Breakout'] = False
    if bbu_col in df.columns and bbb_col in df.columns and close_col in df.columns:
        # Check if bandwidth is low (e.g., in bottom N percentile of its history)
        bandwidth_history = df[bbb_col].dropna()
        if len(bandwidth_history) > BBANDS_PERIOD: # Need some history to compare
            bandwidth_threshold = np.percentile(bandwidth_history, BBANDS_BANDWIDTH_PCT_THRESHOLD)
            # Check if recent bandwidth was low AND current close breaks upper band
            # Look back slightly for low bandwidth condition
            recent_low_bw = (df[bbb_col].rolling(window=5).mean().iloc[-2] < bandwidth_threshold) if len(df)>5 else False # Avg BW over last 5 days (excluding today) was low
            
            if recent_low_bw and last[close_col] > last[bbu_col]:
                signals['Bollinger_Squeeze_Breakout'] = True
                triggered_count += 1

    return signals, triggered_count


# --- Core Screening Function ---

def compute_metrics_and_signals(ticker_symbol):
    """
    For a given ticker, compute volatility metrics, technical indicators,
    check filter conditions, and reversal signals.
    """
    try:
        ticker_symbol = ticker_symbol.strip().upper()
        if not ticker_symbol: return None
        
        stock = yf.Ticker(ticker_symbol)
        
        # 1. Get Historical Data (need enough for TAs and historical RV)
        price_history = stock.history(period=HISTORY_PERIOD)
        if price_history.empty or len(price_history) < max(SLOW_SMA, RV_WINDOW, BBANDS_PERIOD) + 5: # Need buffer
            # print(f"Insufficient price history for {ticker_symbol}")
            return None
            
        # Get current price
        underlying_price = get_current_price(price_history)
        if np.isnan(underlying_price):
             # print(f"Could not determine current price for {ticker_symbol}")
             return None

        # 2. Basic Liquidity Check
        avg_volume = price_history['Volume'].rolling(window=30).mean().iloc[-1]
        pass_volume = avg_volume >= MIN_AVG_VOLUME
        if not pass_volume: # Fail fast if illiquid
             # print(f"Skipping {ticker_symbol} due to low volume ({avg_volume:.0f})")
             return {'ticker': ticker_symbol, 'pass_filters': False, 'reason': 'Low Volume'}


        # 3. Check Options Availability & Dates
        try:
            all_exps = list(stock.options)
            if not all_exps:
                # print(f"No options found for {ticker_symbol}")
                return {'ticker': ticker_symbol, 'pass_filters': False, 'reason': 'No Options'}
        except Exception as e:
             # print(f"Error accessing options dates for {ticker_symbol}: {e}")
             return {'ticker': ticker_symbol, 'pass_filters': False, 'reason': 'Options Error'}

        today = datetime.today().date()
        all_exps_dates = sorted([datetime.strptime(d, "%Y-%m-%d").date() for d in all_exps])
        
        # Find expiration closest to 30 days for IV/RV calculation
        target30_date = min(all_exps_dates, key=lambda d: abs((d - today).days - 30))
        target30_str = target30_date.strftime("%Y-%m-%d")
        dte_target30 = (target30_date - today).days

        # 4. Calculate Volatility Metrics
        #   a) Realized Volatility (Current 30-day)
        rv30 = yang_zhang(price_history, window=RV_WINDOW, return_series=False)
        if np.isnan(rv30) or rv30 == 0:
             # print(f"Could not calculate RV30 for {ticker_symbol}")
             return {'ticker': ticker_symbol, 'pass_filters': False, 'reason': 'RV Calc Error'}

        #   b) Implied Volatility (ATM near 30 days)
        iv_target30 = get_atm_iv_for_exp(stock, target30_str, underlying_price)
        if iv_target30 is None:
             # print(f"Could not get IV30 for {ticker_symbol}")
             return {'ticker': ticker_symbol, 'pass_filters': False, 'reason': 'IV Calc Error'}

        #   c) IV / RV Ratio
        iv30_rv30_ratio = iv_target30 / rv30

        #   d) Historical Realized Volatility (for IV Percentile Proxy)
        historical_rv = yang_zhang(price_history, window=RV_WINDOW, return_series=True).dropna()
        if historical_rv.empty:
            # print(f"Could not calculate historical RV for {ticker_symbol}")
            iv_percentile = np.nan # Cannot calculate percentile
        else:
            # Compare current IV30 to the historical distribution of RV30
            iv_percentile = percentileofscore(historical_rv, iv_target30, kind='strict') # Pct of historical RV values < current IV

        #   e) (Optional) Term Structure Slope (as informational)
        #      Find nearest expiry and one 4-20 days out
        first_expiry = all_exps_dates[0]
        first_exp_str = first_expiry.strftime("%Y-%m-%d")
        dte_first = (first_expiry - today).days
        iv_first = get_atm_iv_for_exp(stock, first_exp_str, underlying_price)

        filtered_exps = [d for d in all_exps_dates if 4 <= (d - today).days <= 20]
        second_expiry = filtered_exps[0] if filtered_exps else None
        ts_slope = np.nan
        dte_second = np.nan
        iv_second = np.nan
        if second_expiry and iv_first is not None:
            second_exp_str = second_expiry.strftime("%Y-%m-%d")
            dte_second = (second_expiry - today).days
            iv_second = get_atm_iv_for_exp(stock, second_exp_str, underlying_price)
            if iv_second is not None and dte_second > dte_first: # Avoid division by zero/negative
                 ts_slope = (iv_second - iv_first) / (dte_second - dte_first)

        # 5. Cheap Volatility Filter (Stage 1)
        pass_cheap_vol_ratio = iv30_rv30_ratio <= IV_RV_THRESHOLD
        pass_cheap_vol_percentile = (not np.isnan(iv_percentile)) and (iv_percentile <= IV_PERCENTILE_THRESHOLD)
        pass_cheap_vol = pass_cheap_vol_ratio or pass_cheap_vol_percentile
        
        if not pass_cheap_vol:
            return {'ticker': ticker_symbol, 'pass_filters': False, 'reason': 'Volatility Not Cheap',
                    'raw': {'avg_volume': avg_volume, 'iv30_rv30': iv30_rv30_ratio, 'iv_percentile': iv_percentile, 'rv30':rv30, 'iv_target30': iv_target30}}

        # 6. Calculate Technical Indicators
        price_history = calculate_technical_indicators(price_history)
        if price_history.empty or price_history.iloc[-1].isna().any(): # Check if latest row has NaNs from TAs
            # print(f"TA calculation failed or resulted in NaNs for {ticker_symbol}")
            return {'ticker': ticker_symbol, 'pass_filters': False, 'reason': 'TA Calc Error'}


        # 7. Reversal Probability Filter (Stage 2)
        reversal_signals, num_reversal_signals = check_reversal_signals(price_history)
        pass_reversal_prob = num_reversal_signals > 0

        if not pass_reversal_prob:
            return {'ticker': ticker_symbol, 'pass_filters': False, 'reason': 'No Reversal Signal',
                     'raw': {'avg_volume': avg_volume, 'iv30_rv30': iv30_rv30_ratio, 'iv_percentile': iv_percentile, 'rv30':rv30, 'iv_target30': iv_target30, 'num_signals': 0}}

        # 8. Passed Both Filters - Calculate Score
        # Score Component 1: Volatility cheapness (lower percentile is better)
        # Normalize percentile to 0-1 range where 1 is best (0th percentile)
        vol_score = (100.0 - iv_percentile) / 100.0 if not np.isnan(iv_percentile) else 0.0
        # If IV percentile failed but ratio passed, give partial score based on ratio? Let's keep it simple for now.

        # Score Component 2: Reversal signals (more is better)
        # Normalize signal count to 0-1 range
        max_signals = 5 # Total number of signal types checked
        signal_score = num_reversal_signals / max_signals

        # Composite Score (simple average, could be weighted)
        composite_score = (vol_score + signal_score) / 2.0

        # 9. Calculate Expected Move (Informational) - Use near-term ATM straddle
        expected_move = "N/A"
        if iv_first is not None:
             try:
                 chain_first = stock.option_chain(first_exp_str)
                 calls = chain_first.calls
                 puts = chain_first.puts
                 calls['strike'] = pd.to_numeric(calls['strike'], errors='coerce')
                 puts['strike'] = pd.to_numeric(puts['strike'], errors='coerce')
                 calls = calls.dropna(subset=['strike'])
                 puts = puts.dropna(subset=['strike'])

                 if not calls.empty and not puts.empty:
                     calls['abs_diff'] = (calls['strike'] - underlying_price).abs()
                     puts['abs_diff'] = (puts['strike'] - underlying_price).abs()
                     call_idx = calls['abs_diff'].idxmin()
                     put_idx = puts['abs_diff'].idxmin()

                     # Use mid-point of bid/ask for straddle price
                     call_bid = calls.loc[call_idx, 'bid']
                     call_ask = calls.loc[call_idx, 'ask']
                     put_bid = puts.loc[put_idx, 'bid']
                     put_ask = puts.loc[put_idx, 'ask']
                     
                     # Handle zero bid/ask often seen for some options
                     call_mid = (call_bid + call_ask) / 2.0 if call_bid > 0 and call_ask > 0 else calls.loc[call_idx, 'lastPrice']
                     put_mid = (put_bid + put_ask) / 2.0 if put_bid > 0 and put_ask > 0 else puts.loc[put_idx, 'lastPrice']

                     if pd.notna(call_mid) and pd.notna(put_mid) and underlying_price > 0:
                         straddle = call_mid + put_mid
                         expected_move = f"{round(straddle / underlying_price * 100, 2)}%"
                     else: straddle = np.nan
                 else: straddle = np.nan
             except Exception:
                 straddle = np.nan # Ignore errors in expected move calc
                 expected_move = "Error"


        # Prepare results dictionary
        return {
            'ticker': ticker_symbol,
            'pass_filters': True,
            'score': composite_score,
            'vol_score': vol_score,
            'signal_score': signal_score,
            'num_reversal_signals': num_reversal_signals,
            'reversal_signals': reversal_signals, # Dict of True/False for each signal
            'iv_percentile': iv_percentile,
            'iv30_rv30_ratio': iv30_rv30_ratio,
            'expected_move': expected_move,
            'raw': {
                'avg_volume': avg_volume,
                'iv_target30': iv_target30,
                'rv30': rv30,
                'ts_slope': ts_slope,
                'underlying_price': underlying_price,
                # Add last values of key indicators for context
                'RSI': price_history[f'RSI_{RSI_PERIOD}'].iloc[-1],
                'STOCH_K': price_history[f'STOCHk_{STOCH_K}_{STOCH_D}_{STOCH_SMOOTH_K}'].iloc[-1],
                'MACD_Hist': price_history[f'MACDh_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}'].iloc[-1],
                'SMA_Fast': price_history[f'SMA_{FAST_SMA}'].iloc[-1],
                'SMA_Slow': price_history[f'SMA_{SLOW_SMA}'].iloc[-1],
                'BB_PctB': price_history[f'BBP_{BBANDS_PERIOD}_{BBANDS_STD}'].iloc[-1],
                'BB_Width': price_history[f'BBB_{BBANDS_PERIOD}_{BBANDS_STD}'].iloc[-1],
            }
        }

    except Exception as e:
        print(f"ERROR processing {ticker_symbol}: {e}")
        # In production, log the error details (traceback)
        import traceback
        # print(traceback.format_exc())
        return {'ticker': ticker_symbol, 'pass_filters': False, 'reason': f'Unhandled Error: {e}'}

# --- Main Screening Script ---

def main():
    tickers_to_screen = get_sp500_tickers()
    # Optional: Use a smaller list for testing
    # tickers_to_screen = ['AAPL', 'MSFT', 'GOOGL', 'NVDA', 'AMD', 'TSLA', 'SPY', 'QQQ', 'IWM', 'NFLX', 'COIN', 'AMC', 'GME'] # Example list
    
    results = []
    failed_tickers = []

    print(f"Screening {len(tickers_to_screen)} tickers based on Volatility and Reversal Signals...\n")
    
    count = 0
    for ticker in tickers_to_screen:
        count += 1
        print(f"Processing {ticker}... ({count}/{len(tickers_to_screen)})")
        
        metrics = compute_metrics_and_signals(ticker)

        if metrics is None:
            print(f"  Data error or insufficient data for {ticker}. Skipping.\n")
            failed_tickers.append({'ticker': ticker, 'reason': 'Data Error/Insufficient'})
            continue
            
        if metrics['pass_filters']:
            results.append(metrics)
            print(f"  >>> PASSED Filters: {ticker} <<<")
            print(f"      Score: {metrics['score']:.3f} (Vol: {metrics['vol_score']:.2f}, Signal: {metrics['signal_score']:.2f})")
            print(f"      IV Pctile: {metrics['iv_percentile']:.1f}%, IV/RV Ratio: {metrics['iv30_rv30_ratio']:.2f}")
            print(f"      Signals ({metrics['num_reversal_signals']}): {[sig for sig, triggered in metrics['reversal_signals'].items() if triggered]}")
            print(f"      Exp. Move: {metrics['expected_move']}")
            print(f"      Price: {metrics['raw']['underlying_price']:.2f}, Vol: {metrics['raw']['avg_volume']:.0f}\n")
        else:
            failed_tickers.append(metrics) # Store failure reason
            print(f"  Skipped {ticker}: {metrics.get('reason', 'Unknown')}\n")

    # Sort the survivors by score (highest first)
    results.sort(key=lambda x: x['score'], reverse=True)

    # Final summary:
    print("\n" + "="*50)
    print("Final Screening Results:")
    print("="*50 + "\n")

    if not results:
        print("No tickers passed both the Cheap Volatility and Reversal Probability filters.")
    else:
        print(f"--- {len(results)} Tickers Passed Filters (Ranked by Score) ---")
        for i, result in enumerate(results):
            print(f"{i+1}. {result['ticker']}")
            print(f"   Score: {result['score']:.3f} (Vol: {result['vol_score']:.2f}, Signal: {result['signal_score']:.2f})")
            print(f"   IV Pct (proxy): {result['iv_percentile']:.1f}%, IV/RV Ratio: {result['iv30_rv30_ratio']:.2f}")
            print(f"   Signals ({result['num_reversal_signals']}): {[sig for sig, triggered in result['reversal_signals'].items() if triggered]}")
            print(f"   Exp. Move: {result['expected_move']}")
            print(f"   Price: {result['raw']['underlying_price']:.2f}, Avg Vol: {result['raw']['avg_volume']:.0f}")
            # Optional: Print more raw data if needed
            # print(f"   Raw IV30: {result['raw']['iv_target30']:.3f}, Raw RV30: {result['raw']['rv30']:.3f}")
            # print(f"   RSI: {result['raw']['RSI']:.1f}, StochK: {result['raw']['STOCH_K']:.1f}, MACD Hist: {result['raw']['MACD_Hist']:.3f}")
            print("-" * 20)

    # Optional: Print summary of failures
    print(f"\n--- {len(failed_tickers)} Tickers Failed Filters ---")
    # Count failure reasons
    from collections import Counter
    failure_counts = Counter(item.get('reason', 'Unknown') for item in failed_tickers)
    print("Failure Reasons:")
    for reason, count in failure_counts.most_common():
        print(f"  - {reason}: {count}")

    print("\n" + "="*50)
    print("Screening Complete.")
    print("="*50)
    
if __name__ == "__main__":
    main()

Screening 503 tickers based on Volatility and Reversal Signals...

Processing MMM... (1/503)
  Skipped MMM: Volatility Not Cheap

Processing AOS... (2/503)
  Skipped AOS: Volatility Not Cheap

Processing ABT... (3/503)
  Skipped ABT: No Reversal Signal

Processing ABBV... (4/503)
  Skipped ABBV: No Reversal Signal

Processing ACN... (5/503)
  Skipped ACN: No Reversal Signal

Processing ADBE... (6/503)
  >>> PASSED Filters: ADBE <<<
      Score: 0.361 (Vol: 0.52, Signal: 0.20)
      IV Pctile: 47.7%, IV/RV Ratio: 0.63
      Signals (1): ['MACD_Histogram_Flip']
      Exp. Move: 0.94%
      Price: 349.44, Vol: 4819225

Processing AMD... (7/503)
  Skipped AMD: Volatility Not Cheap

Processing AES... (8/503)
  Skipped AES: Volatility Not Cheap

Processing AFL... (9/503)
  Skipped AFL: Volatility Not Cheap

Processing A... (10/503)
  Skipped A: Volatility Not Cheap

Processing APD... (11/503)
  Skipped APD: Low Volume

Processing ABNB... (12/503)
  Skipped ABNB: Volatility Not Cheap

Process